### Финальный проект. Step 3.

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [10]:
X_test[['age', 'years_of_experience', 'lesson_price', 'qualification', 'physics', 'chemistry', 'biology', 'mean_exam_points']].head(3)

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,mean_exam_points
0,59.0,0.0,2200.0,3.0,0.0,1.0,0.0,85.0
1,46.0,0.0,1250.0,1.0,1.0,0.0,0.0,53.0
2,53.0,1.0,1250.0,1.0,0.0,0.0,0.0,52.0


In [11]:
import urllib.request
import json      

def get_prediction(age, years_of_experience, lesson_price, qualification, physics, chemistry, biology, mean_exam_points, 
                  lesson_price_pow, mean_exam_points_pow):
    body = {'age': age, 
            'years_of_experience': years_of_experience,
            'lesson_price': lesson_price,
            'qualification': qualification,
            'physics': physics,
            'chemistry': chemistry,
            'biology': biology,
            'mean_exam_points': mean_exam_points,
            'lesson_price_pow': lesson_price_pow,
            'mean_exam_points_pow': mean_exam_points_pow} 

    myurl = "http://172.20.10.2:8180/predict" ## Поставить здесь айпи адрес на котором поднимется питоновский файл
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [12]:
get_prediction(42, 4, 2500, 3, 1, 1, 1, 80, 1, 1)

0.13405884908425184

In [13]:
features = ['lesson_price', 'mean_exam_points']

In [14]:
for feature in features:
    X_test[feature + '_pow'] = 1/(np.power(X_test[feature], 2) + 1)

In [15]:
%%time
predictions = X_test[['age', 'years_of_experience', 'lesson_price', 'qualification', 'physics', 'chemistry', 'biology', 
                      'mean_exam_points', 'lesson_price_pow', 'mean_exam_points_pow']].iloc[:500].apply(
              lambda x: get_prediction(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9]), 1)

Wall time: 8.4 s


In [16]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.8818774131274132